# The Malthusian Growth Model

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **model analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `modelproject.py` file includes a function which could be used multiple times in this notebook.

Imports and set magics:

In [1]:
import numpy as np
from scipy import optimize
import sympy as sm

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import modelproject

# Description of the Malthusian Model

The Malthusian Growth Model is a model of population growth.

We consider a model where time is discrete, t=0,1,2,...,$\infty$. The economy is closed meaning that total production is equal to total income. There are three main assumptions to the Malthusain model. The first assumption is that labour is subject to diminishing marginal returns in production, which is represented in the model as the Cobb-Douglas production function:

$Y_t=L_t^{1-\alpha}(AX)^\alpha, 0 < \alpha < 1$,

where t denotes time, t=0,1,2.., $Y_t$ denotes output, A is the level of techonology, $L_t$ is labor input and X is land (fixed factor of production). Output per worker can we written as:

$y_t=(\frac{AX}{L_t})^\alpha$

This equation shows that output per worker declines as labor (L) increases due to the assumption of deminishing returns, given that AX is constant. In our model we will assume that the level of technology (A) and land (X) is constant. The assumption of A being constant will be discussed later.

The next main assumption of the Malthusian model is that the birth rate in the economy increases with income per capita, which is shown in this equation:

$n_t=\eta y_t$,

where $\eta$ is a positive parameter. This equation implies that every person in the economy gets $n_t$ children in each period. We further assume that the preferable number of children increases with income per capita. With this assumption it would be reasonable to think of $\eta$ as a parameter below one, where each parent in the economy uses a fixed fraction of their income to provide for their children (food, clothes, etc.). We can thus interpret $\eta$ as a parameter capturing preferences for family size and cost of child rearing etc.

For simplicity, we assume that everyone in the economy works, meaning no unemployment. We then assume that the labor force ($L_t$) evolves as follows:

$L_{t+1}=n_tL_t+(1-\mu)L_t, L_0$ given.

This equations implies that the labor force in the next period ($L_{t+1}$) is equal to the current labor force ($L_t$) times births per person ($n_t$), plus the part of the current labor force, which is still in the labor force (the fraction $(1-\mu)$). Assuming that people work intil they die, we can think of $\mu$ as the mortality rate, where a higher $\mu$ means higher mortality.

## Analytical solution

If your model allows for an analytical solution, you should provide here.

You may use Sympy for this. Then you can characterize the solution as a function of a parameter of the model.

To characterize the solution, first derive a steady state equation as a function of a parameter using Sympy.solve and then turn it into a python function by Sympy.lambdify. See the lecture notes for details. 

## Numerical solution

You can always solve a model numerically. 

Define first the set of parameters you need. 

Then choose one of the optimization algorithms that we have gone through in the lectures based on what you think is most fitting for your model.

Are there any problems with convergence? Does the model converge for all starting values? Make a lot of testing to figure these things out. 

# Further analysis

Make detailed vizualizations of how your model changes with parameter values. 

Try to make an extension of the model. 

# Conclusion

Add concise conclusion. 